In [ ]:
%pylab inline

In [ ]:
from lsst.eo_utils.base.butler_utils import get_butler_by_repo

from lsst.eo_utils.base.image_utils import get_ccd_from_id,\
    get_amp_list,\
    get_dimension_arrays_from_ccd, get_image_frames_2d,\
    get_geom_regions, get_raw_image

from lsst.eo_utils.base.plot_utils import FigureDict

import lsst.eotest.image_utils as imutil

import matplotlib.pyplot as plt

In [ ]:
def get_bias_image(ccd, amp, bias_method):
    """Construct the bias image for one amp

    @param butler (Butler)   The data butler
    @param ccd (MaskedCCD)   The ccd object
    @param amp (int)         Amplifier index
    @param bias_method (str) 
    
    @returns (tuple)
       bim (ImageF)          The bias image
       regions (dict)        Dictionary with region boxes
    """
    regions = get_geom_regions(ccd, amp)
    serial_oscan = regions['serial_overscan']
    im = get_raw_image(ccd, amp)
    bim = imutil.bias_image(im, serial_oscan, bias_method=bias_method)
    return bim, regions

In [ ]:
def plot_bias_v_row(butler, ccd, figs, bias_type):
    """Plot the bias versus row for all the amps on a ccd

    @param butler (Butler)   The data butler
    @param ccd (MaskedCCD)   The ccd object
    @param figs (FigureDict) Figure bookkeeping object
    @param bias_type (str)   Method used to compute bias
    """

    # Get the list of amp indices
    amps = get_amp_list(ccd)

    # Get the dimensions for making the plot 
    # in this case the rows in the serial overscan region    
    x_arrays = get_dimension_arrays_from_ccd(ccd)
    x_array = x_arrays['row_s']

    # Loop on the amps
    for i, amp in enumerate(amps):
        # Get the bias image for each version
        bim, regions = get_bias_image(ccd, amp, bias_type)
        s_oscan = regions['serial_overscan']
        
        # Get the row-by-row means
        # account of the readout order, in this case 
        # we want to account for the step direction in x 
        # changing
        step = regions['step_x']
        bim_row_mean = bim[s_oscan].getArray().mean(1)[::step]
    
        # Add a plot of the row-by-row means versus row number to the figure "val"
        # in the ith slot
        figs.plot("val", i, x_array[0:len(bim_row_mean)], bim_row_mean)

In [ ]:
# Set the path to the Butler repository and import a Butler
butler = get_butler_by_repo('ts8')

In [ ]:
# Get a dataId for the first bias frame for a FE55 run
incompleteDataId = dict(run='6006D', imageType='BIAS', testType='FE55', detectorName='S00')
ss = butler.subset('raw', '', incompleteDataId)
dataId = ss.cache[0]

# Specify the corresponding file
fname = butler.get('raw_filename', dataId)[0][0:-3]

In [ ]:
# Get the data using the Butler, and using MaskedCCD
exp = get_ccd_from_id(butler, ss.cache[0], [])
ccd = get_ccd_from_id(None, fname, [])

In [ ]:
# Setup a place to store the figures
figs = FigureDict()
# This will add a single 4x4 array of figures, under the key "val"
figs.setup_amp_plots_grid("val", title="Bias by row", xlabel="row", ylabel="Magnitude [ADU]")

# specify how to make the bias image
bias_type = 'spline'

# make the plots for both versions
plot_bias_v_row(butler, exp, figs, bias_type)
plot_bias_v_row(None, ccd, figs, bias_type)

In [ ]:
plt.show()